# prompt Optimization Techniques

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [9]:
async def generate_llm_response(prompt):
    """
    Generates a response using the llm and prompt
    Args:
     prompt (str): The input prompt
    Returns:
        str: The generated response
    """
    response = await llm.ainvoke(prompt)
    return response

#A/B Testing prompt

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_a = PromptTemplate(
    input_variables=["topic"],
    template="Explain the topic '{topic}' in simple terms"

)

prompt_b = PromptTemplate(
    input_variables=["topic"],
    template="Provide a friendly explanation of '{topic}', including key concepts and examples"
)

In [11]:
import nest_asyncio
nest_asyncio.apply()
import asyncio

import re
import numpy as np

async def evaluate_response(response, criteria):
    """
    Evaluates the prompt and assigns score based on the criteria
    Args:
        response (str): The response generated from the llm
        criteria (list): List of criteria to evaluate on
    Returns:
        float: the average score across all criteria 
    """
    scores = []
    for criterion in criteria:
        print(f"Evaluating response based on {criterion}...")
        prompt = f"On a scale of 1 to 10 rate the following response on {criterion}. Start your response with neumeric score:\n\n{response}"
        response = asyncio.run(generate_llm_response(prompt)).content
        score_match = re.search(r"\d+", response)
        if score_match:
            score = int(score_match.group())
            scores.append(min(score, 10))
        else:
             print(f"Warning: Could not extract numeric score for {criterion}. Using default score of 5.")
             scores.append(5)

    return np.mean(scores)

In [ ]:
topic = "machine learning"
response_a = asyncio.run(generate_llm_response(prompt_a.format(topic=topic))).content
response_b = asyncio.run(generate_llm_response(prompt_b.format(topic=topic))).content

In [13]:
criteria = ["clarity", "informativeness", "engagement"]
score_a = asyncio.run(evaluate_response(response_a, criteria))
score_b = asyncio.run(evaluate_response(response_b, criteria))

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...


In [14]:
print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

Prompt A score: 9.00
Prompt B score: 8.00
Winning prompt: A


#Iterative Refinement

In [15]:
async def refining_prompt(initial_prompt, topic, iterations=3):
    """
    Refines a prompt through multiple iterations
    Args:
        initial_prompt (str): The starting prompt template
        topic (str): The topic to explain
        iterations (int): Number of refinement iterations.

    Returns:
        PromptTemplate: The final refined prompt template.
    """
    current_prompt = initial_prompt
    for i in range(iterations):
        try:
            response = asyncio.run(generate_llm_response(current_prompt.format(topic=topic))).content
        except KeyError as e:
            print(f"Error in iteration {i+1}: Missing key {e}. Adjusting prompt...")
            current_prompt.template = current_prompt.template.replace(f"{{{e.args[0]}}}", "relevant example")
            response = asyncio.run(generate_llm_response(current_prompt.format(topic=topic))).content
        
        feedback_prompt = f"Analyze the following explanation of '{topic}' and suggest improvements to the prompt generated it:\n\n{response}"
        feedback = asyncio.run(generate_llm_response(feedback_prompt)).content

        refine_prompt = f"Based on the feedback: '{feedback}', improve the following prompt template. Ensure to only use the variable {{topic}} in your prompt:\n\n{current_prompt.template}"
        refined_template = asyncio.run(generate_llm_response(refine_prompt)).content

        current_prompt = PromptTemplate(
            input_variables=["topic"],
            template=refined_template
        )
        print(f"Iteration {i+1} prompt: {current_prompt.template}")

    return current_prompt

In [21]:
def llm_response(prompt):
    response = asyncio.run(generate_llm_response(prompt)).content
    return response

def eval_response(prompt, criteria):
    score = asyncio.run(evaluate_response(prompt, criteria))
    return score

In [18]:
topic = "machine learning"
response_a = llm_response(prompt_a.format(topic=topic))
response_b = llm_response(prompt_b.format(topic=topic))

In [22]:
criteria = ["clarity", "informativeness", "engagement"]
score_a = eval_response(response_a, criteria)
score_b = eval_response(response_b, criteria)

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...


C:\Users\user\AppData\Local\Temp\ipykernel_4224\1901663314.py:2: RuntimeWarning: coroutine 'evaluate_response' was never awaited
  score_a = eval_response(response_a, criteria)


Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...


C:\Users\user\AppData\Local\Temp\ipykernel_4224\1901663314.py:3: RuntimeWarning: coroutine 'evaluate_response' was never awaited
  score_b = eval_response(response_b, criteria)


In [23]:
print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

Prompt A score: 8.00
Prompt B score: 8.33
Winning prompt: B


In [26]:
initial_prompt = prompt_b if score_b > score_a else prompt_a
refined_prompt = asyncio.run(refining_prompt(initial_prompt, "machine learning"))

Iteration 1 prompt: Here is the improved prompt template:

**{topic}: A Beginner's Guide**

{topic} is a type of artificial intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed. It's a powerful technology that has many applications in fields such as healthcare, finance, and education.

**Key Concepts:**

* **Training Data**: The data used to teach the {topic} model what to learn.
* **Model**: The algorithm that learns from the training data.
* **Algorithm**: A set of instructions that the model follows to learn from the training data.
* **Features**: The characteristics or attributes of the data that the model uses to make predictions.
* **Labels**: The correct answers or outcomes that the model is trying to predict.
* **Accuracy**: A measure of how well the model performs on new, unseen data.

**Examples:**

* **[Insert example 1]**: A {topic} model can be trained to [insert brief description of example 1].
* **[I

In [27]:
print("\nFinal refined prompt:")
print(refined_prompt.template)


Final refined prompt:
Here is the improved prompt template incorporating the suggestions provided:

**Machine Learning: A Beginner's Guide to {topic}**

Machine learning is a type of artificial intelligence that enables computers to learn from data and make predictions or decisions without being explicitly programmed. It's a powerful technology that has many applications in fields such as healthcare, finance, and education.

**Key Concepts:**

* **Supervised Learning**: A type of machine learning where the model is trained on labeled data to make predictions on new, unseen data.
* **Unsupervised Learning**: A type of machine learning where the model is trained on unlabeled data to identify patterns and relationships.
* **Deep Learning**: A type of machine learning that uses neural networks to analyze data and make predictions.
* **Data Preparation**: This section covers training data, features, and labels, grouping related concepts together.

**Examples:**

* **Healthcare:** Machine l

#Comparision

In [28]:
original_response = llm_response(initial_prompt.format(topic="machine learning"))
refined_response = llm_response(refined_prompt.format(topic="machine learning"))

original_score = eval_response(original_response, criteria)
refined_score = eval_response(refined_response, criteria)

print(f"Original prompt score: {original_score:.2f}")
print(f"Refined prompt score: {refined_score:.2f}")
print(f"Improvement: {(refined_score - original_score):.2f} points")

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Original prompt score: 8.00
Refined prompt score: 9.00
Improvement: 1.00 points
